In [108]:
import numpy as np
import random
import pickle
import gzip
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml


def cost_grad(a , y):
    return (a-y)

def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

def sig_diff(z):
    return sigmoid(z)*(1-sigmoid(z))


class Network():
    def __init__(self , sizes):
        self.sizes = sizes
        self.num_layers = len(sizes)
        self.biases = [np.random.randn(y , 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x) for y,x in zip(sizes[1:] , sizes[:-1])]
        
    def feedforward(self , a):
        for b,w in zip(self.biases, self.weights):
            a = np.matmul(w , a) + b
        return a
    def SGD(self, training_data, epochs, mini_batch_size, eta, lmbda = 5.0,evaluation_data=None):
        if evaluation_data: n_data = len(evaluation_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:mini_batch_size + k] for k in range(0, n , mini_batch_size)]
            for mini_batch in mini_batches:
                self.update(mini_batch , lmbda , eta , n)  
            print("Epoch {} completed!!!".format(j))
      
        accuracy = self.accuracy(training_data)
        return accuracy
    
    def update(self ,mini_batch, lmbda ,eta, n ):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x,y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x,y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

        self.weights = [(1-eta*lmbda/n)*w - (eta/len(mini_batch))*nw for w,nw in zip(self.weights, delta_nabla_w)]
        self.biases = [b - (eta/len(mini_batch))*nb for b,nb in zip(self.biases,delta_nabla_b)]

    def accuracy(self, data):
        results = [(np.argmax(self.feedforward(x)), y) for (x, y) in data]
        return sum(int(x == y) for (x, y) in results)

        
    def backprop(self, x , y ):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        zs = []
        x = np.reshape(x , (784,1))
        acti = x
        #print("acti:",acti.shape)
        
        activations = [x]
        for b,w in zip(self.biases , self.weights):
            #print("w: " , w.shape)
            #print("x: " , x.shape)
            #print("1: ",np.matmul(w,acti).shape)
            #print("b: ",b.shape)
            z = np.add(np.matmul(w,acti) , b)
            #print("z:",z.shape)
            zs.append(z)
            acti = sigmoid(z)
            activations.append(acti)
            
        delta = np.multiply(cost_grad(zs[-1] , y)  , sig_diff(zs[-1]))
        #print(cost_grad(zs[-1] , y).shape)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for i in range(2 , self.num_layers):
#             delta = np.multiply(np.matmul(nabla_w[-i+1].transpose() , delta) , sig_diff(zs[-i]))
#             nabla_b = delta
#             nabla_w = np.dot(delta , activations[-i-1].transpose())
            delta = np.multiply(np.matmul(self.weights[-i+1].transpose(), delta) , sig_diff(zs[-i]))
            #print(delta.shape)
            nabla_b[-i] = delta
            nabla_w[-i] = np.dot(delta, activations[-i-1].transpose())
        return (nabla_b , nabla_w)

    def default_weight_initializer(self):
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x) for x, y in zip(self.sizes[:-1], self.sizes[1:])]

In [110]:
from scipy.io import loadmat
mnist = loadmat("D:/Acads(sem-wise)/Some ML/data/mnist-original")
x = mnist["data"].T
y = mnist["label"][0]
random.shuffle(x)
random.shuffle(y)
x_train, x_test = x[:400], x[400:500]
y_train, y_test = y[:400], y[400:500]


training_data = [[x/255,y] for x,y in zip(x_train,y_train)]
test_data = [ [x,y] for x,y in zip(x_test,y_test)]
#print(training_data[0])
net = Network([784, 30, 10])
net.default_weight_initializer()
print(net.SGD(training_data, 30, 10, 0.1, lmbda = 5.0,evaluation_data=test_data))

Epoch 0 completed!!!
Epoch 1 completed!!!
Epoch 2 completed!!!
Epoch 3 completed!!!
Epoch 4 completed!!!
Epoch 5 completed!!!
Epoch 6 completed!!!
Epoch 7 completed!!!
Epoch 8 completed!!!
Epoch 9 completed!!!
Epoch 10 completed!!!
Epoch 11 completed!!!
Epoch 12 completed!!!
Epoch 13 completed!!!
Epoch 14 completed!!!
Epoch 15 completed!!!
Epoch 16 completed!!!
Epoch 17 completed!!!
Epoch 18 completed!!!
Epoch 19 completed!!!
Epoch 20 completed!!!
Epoch 21 completed!!!
Epoch 22 completed!!!
Epoch 23 completed!!!
Epoch 24 completed!!!
Epoch 25 completed!!!
Epoch 26 completed!!!
Epoch 27 completed!!!
Epoch 28 completed!!!
Epoch 29 completed!!!
0
